In [ ]:
# Colab only: Run this cell to download/install packages
import sys
if 'google.colab' in sys.modules:
    !curl http://www.datasciencecourse.org/assignments/hw3_linear.tar.gz --output hw3_linear.tar.gz
    !tar -xzf hw3_linear.tar.gz
    !mv hw3_linear/* ./
    !pip install --upgrade --no-deps --force-reinstall git+https://github.com/locuslab/mugrade.git

# Linear Regression

In this homework, we are going to apply linear regression to the problem of predicting developer satisfaction based upon information about their careers, from a StackOverflow survey.  The data from this question is based on the [2019 StackOverflow Survey](https://insights.stackoverflow.com/survey/2019); accordingly, the subset bundled with this assignment is also released under the Open Database License (ODbL) v1.0.  For this problem, you should not use Scikit-Learn, but instead, implement all the least squares solutions manually.


### Q1 Data Parsing

The data from this survey has the values as described below.  Your eventual goal will be to predict (a numerical equivalent of) the `CarreerSat` column, indicating how satisfied the subject is with their career), based upon the other values.  Note that because the career satisfaction values are ordinal, we likely should not be predicting them with linear regression, but as an illustrative example, this is still a reasonable task.

The data set contains the following columns.

| Column | Sample | Does/is the respondent... | Type/Values |
| --- |:--- |:--- |:--- |
| **CareerSat** | 'vs' | satisfied with their career? | (`vd`, `sd`, `ne`, `NA`, `ss`, `vs`) -- corresponding to ({very, slightly}, {satisfied, dissatisfied}), neutral, and not applicable |
| MgrWant | 'n' | ...want to be a manager? | boolean |
| Age    | '22'   | age | integer     |
| CodeRevHrs | '2' | hours a week spent reviewing code | integer |
| ConvertedComp | '61000' | yearly compensation in 2019 USD | integer |
| Country | 'United States' | lives in country | string _(ignore in regression)_ |
| Dependents | 'n' | ...have children or other dependents. | boolean |
| DevEnvironVSC | 'y' | ...use Visual Studio Code | boolean |
| DevTypeFullStack | 'n' | ...identify as a full-stack developer | boolean |
| EdLevel | 'bachelors' | maximum education level | (`other`, `bachelors`, `masters`, `doctoral`) |
| EduOtherMOOC | 'y' | ...ever taken a Massively Open Online Course | boolean |
| EduOtherSelf | 'y' | ...ever taught themselves a new platform | boolean _(ignore in regression, this is  duplicate field)_ |
| Extraversion | 'y' | ...prefer in-person meetings to online meetings | boolean |
| GenderIsMan | 'y' | ...male | boolean |
| Hobbyist | 'n' | ...write code as a hobby? | boolean |
| MgrIdiot | 'very' | ...think their manager knows what they are doing? | (`NA`, `not`, `some`, `very`), in order of increasing confidence |
| OpSys | 'win' | which OS do they use? | (`win`, `mac`, `tux`, `NA`), for (Windows, Mac OSX, Linux-like, NA) |
| OpenSourcer | 'Never' | ...contribute to open-source projects? | (`never`, `year`, `month-year`, `month`), in increasing order of frequency |
| OrgSize | '100-499' | number of employees in organization? | (`NA`, `1`, `2-9`, `10-19`, `20-99`, `100-499`, `500-999`, `1,000-4,999`, `5,000-9,999`, `10,000+`) |
| Respondent | '4' | respondent ID from original data | integer _(ignore in regression)_ |
| Student | 'n' | ...currently a student? | boolean |
| UndergradMajorIsComputerScience | 'y' | ...majored in CS? | boolean |
| UnitTestsProcess | 'n' | ...use unit tests in their job? | boolean |
| WorkWeekHrs | '40' | hours a week worked | integer |
| YearsCode | 3 | years since first programming | integer |
| YearsCodePro | 0 | years programming professionally | integer |

When you load the data for linear regression, you will want to convert each type to a floating point value according to the columns above.  Specifically, for each column listed above, convert the value to a numeric quantity using the rules below.  Note that there are some fairly unnatural assumptions here (like converting NAs to 0.0), but these are largely to illustrate the methodology without adding too much additional processing work.

 - _boolean_ : `y`/`NA`/`n` assigned to `+1.0`/`0.0`/`0.0`
 - _integer_ : convert to `float`, preserving value. `NA` equals `0.0`. 
 - _string_ : not included in regression; we'll use it later
 - CareerSat: Map (`vd`, `sd`, `ne`, `NA`, `ss`, `vs`) to (-2.0, -1.0, 0.0, 0.0, 1.0, 2.0)
 - EdLevel: Map (`other`, `bachelors`, `masters`, `doctoral`) to (0.0, 1.0, 1.5, 2.0)
 - MgrIdiot: Map (`NA`, `not`, `some`, `very`) to (-1.0, -1.0, 0.0, 1.0)
 - OpSys: Map (`win`, `mac`, `NA`, `tux`, `BSD`) to (-1.0, 0.0, 0.0, 1.0, 1.0)
 - OpenSourcer : Map (`never`, `year`, `month-year`, `month`) to (0.0, 0.5, 1.0, 2.0)
 - OrgSize: Map each range "$a$-$b$" to the value $ln(a)$. Treat `NA` as `ln(1.0) = 0`. We are converting an exponentially distributed range to a linearly distributed one.

Remove the columns listed above as being ignored. 

Some hints:
  1. Load the csv file with `pd.read_csv(filname, dtype=str, keep_default_na=False)` to ensure that you load all columns as text (so you can do your own preprocessing), and ignore pandas's default conversion to NaN values.
  2. Use the `.apply()` function in pandas to convert

In [150]:
import csv
import gzip
import math
import hashlib
import numpy as np
import pandas as pd
import mugrade
import re
from sklearn.model_selection import train_test_split

In [405]:
# @mugrade.local_tests
# @mugrade.submit_tests('28PY3HJpnZGQT8R2VgdC')

def parse_stackoverflow_data():
    """Load data from the "eggs.csv.gz" file, and convert for use in linear regression
    
    Returns:
        pandas.DataFrame, containing the data converted to floating point values appropriately.
    """
    filename = "eggs.csv.gz"
    data = pd.read_csv(filename, dtype=str, keep_default_na=False)
    bool_col = ['MgrWant', 'Dependents', 'DevEnvironVSC', 'DevTypeFullStack', 'EduOtherMOOC', 'Extraversion',
               'GenderIsMan', 'Hobbyist', 'Student', 'UndergradMajorIsComputerScience', 'UnitTestsProcess']
    int_col = ['Age', 'CodeRevHrs', 'ConvertedComp', 'WorkWeekHrs', 'YearsCode', 'YearsCodePro']
    
    for col in int_col:
        data[col] = pd.to_numeric(data[col], errors='coerce')
        data[col] = data[col].fillna(0.0)
    data['Respondent'] = data['Respondent'].astype('float')
    
    f = lambda x: +1.0 if x == 'y' else 0.0
    for col in bool_col:
        data[col]= data[col].apply(f)
    
    data = data.drop(['Country', 'Respondent', 'EduOtherSelf'], axis = 1)
    data['CareerSat']= data['CareerSat'].map(dict(zip(data.CareerSat.astype('category').cat.categories,[0.0,0.0,-1.0,1.0,-2.0,2.0])))
    data['EdLevel']=data['EdLevel'].map(dict(zip(data.EdLevel.astype('category').cat.categories,[1.0,2.0,1.5,0.0])))
    data['MgrIdiot']=data['MgrIdiot'].map(dict(zip(data.MgrIdiot.astype('category').cat.categories,[-1.0,-1.0,0.0,1.0])))
    data['OpSys']=data['OpSys'].map(dict(zip(data.OpSys.astype('category').cat.categories,[1.0,0.0,0.0,1.0,-1.0])))
    data['OpenSourcer']=data['OpenSourcer'].map(dict(zip(data.OpenSourcer.astype('category').cat.categories,[2.0,1.0,0.0,0.5])))
    
    org_d = dict()
    unique_val =  set(data['OrgSize'])
    for i in unique_val:
        if i == 'NA':
            org_d[i] = 0.0
        else: 
            i = re.sub("[ ,+]", "", i)
            i = i.split('-')[0]
            org_d[i] = np.log(int(i)) 
            
    replace_org = []
    for i in data['OrgSize']:
        if i == 'NA':
            replace_org.append(org_d[i])            
        else:
            i = re.sub("[ ,+]", "", i)
            i = i.split('-')
            replace_org.append(org_d[i[0]])
    data['OrgSize'] = replace_org
    
    return data

    pass

In [411]:
parse_stackoverflow_data().describe(include='all')

,CareerSat,MgrWant,Hobbyist,OpenSourcer,Student,EdLevel,UndergradMajorIsComputerScience,EduOtherMOOC,OrgSize,DevTypeFullStack,...,ConvertedComp,WorkWeekHrs,CodeRevHrs,UnitTestsProcess,DevEnvironVSC,OpSys,Extraversion,Age,GenderIsMan,Dependents
count,65679.000000,65679.000000,65679.000000,65679.000000,65679.000000,65679.000000,65679.000000,65679.000000,65679.000000,65679.000000,...,6.567900e+04,65679.000000,65679.000000,65679.000000,65679.000000,65679.000000,65679.000000,65679.000000,65679.000000,65679.000000
mean,0.949954,0.241630,0.159594,0.634076,0.161635,0.904718,0.667002,0.572070,4.230616,0.293336,...,9.765935e+04,37.290117,3.614195,0.368839,0.524947,-0.196806,0.602552,28.335349,0.888732,0.388161
std,1.170025,0.428074,0.366232,0.634092,0.368118,0.559362,0.471290,0.494782,2.924413,0.455294,...,2.552100e+05,38.733760,5.101685,0.482494,0.499381,0.809142,0.489374,11.937477,0.314467,0.487335
min,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.302585,0.000000,...,3.024000e+03,36.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,24.000000,1.000000,0.000000
50%,1.000000,0.000000,0.000000,0.500000,0.000000,1.000000,1.000000,1.000000,2.995732,0.000000,...,3.838300e+04,40.000000,2.000000,0.000000,1.000000,0.000000,1.000000,28.000000,1.000000,0.000000
75%,2.000000,0.000000,0.000000,1.000000,0.000000,1.500000,1.000000,1.000000,6.907755,1.000000,...,8.336400e+04,42.000000,5.000000,1.000000,1.000000,0.000000,1.000000,34.000000,1.000000,1.000000
max,2.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,9.210340,1.000000,...,2.000000e+06,4850.000000,99.000000,1.000000,1.000000,1.000000,1.000000,99.000000,1.000000,1.000000


### Q2 Splitting Data

Now we prepare the converted data for regression. In this step, we:

 1. Extract the data as a numpy array
 2. Split the data into train and validation sets.  You can use the first 20% of the data (rounded down) as the validation set and keep the remaining as the training set. (Note that it is common practice to randomize the dataset; this has already been done. Don't shuffle the dataset for this assignment.)
 3. Split each set into the predicted column (the first column in the data frame) and the feature columns (the remaining columns), plus a final column corresponding to a constant 1.0 value.  Not that you should keep the ordering of the feature columns the same as they appear in the data.


In [406]:
@mugrade.local_tests
# @mugrade.submit_tests('28PY3HJpnZGQT8R2VgdC')

def split_data(df):
    """split the data into training and validation sets, and convert them to np.ndarray. (Step 1 and 2 above.)

    args:
        df : pandas.DataFrame -- the parsed data, as returned by parse_stackoverflow_data()

    returns: X_train, y_train, X_val, y_val
      X_train  : np.ndarray -- the second 80% of the data features
      y_train : np.ndarray -- the second 80% of the target values
      X_val : np.ndarray -- the first 20% (rounded down) of the data features
      y_val : np.ndarray -- the first 20% of the target values
    """
    data = np.array(df)

    # calculate row position of end of 20% data
    twenty_percent = int(len(data)*.2)
    eighty_percent = len(data)-twenty_percent
    
    # calculate starting row position of 80% data for training
    train_c = len(data) - twenty_percent

    # create validation with 20% data
    X_val = data[:twenty_percent, 1:int(df.shape[1])]
    X_val = np.append(X_val, np.ones([twenty_percent,1]), axis=1)
    
    # create train data with 80% data
    X_train = data[twenty_percent:, 1:int(df.shape[1])]
    X_train = np.append(X_train, np.ones([train_c,1]), axis=1)
    
    # create y_val with 20% data
    y_val = data[:twenty_percent,0]
    
    # create y_train with 80% data
    y_train = data[twenty_percent:,0]
    

    
    return X_train, y_train, X_val, y_val
    
    
    pass

Running local tests for function split_data():
  Test 1 PASSED
  Test 2 PASSED
  Test 3 PASSED


In [418]:
print(result[2])

[[0. 0. 0. ... 1. 0. 1.]
 [0. 0. 2. ... 1. 0. 1.]
 [0. 0. 0. ... 1. 1. 1.]
 ...
 [0. 0. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 1. 0. 1.]
 [1. 0. 0. ... 1. 1. 1.]]


In [352]:
result = split_data(parse_stackoverflow_data())


(52544, 23)


In [354]:
parse_stackoverflow_data().shape

(65679, 23)

In [355]:
52544+13135

65679

In [353]:
print(result[0].shape)
print(result[1].shape)
print(result[2].shape)
print(result[3].shape)


(52544, 23)
(52544,)
(13135, 23)
(13135,)


In [217]:
#     # create np array of ones for training 
#     train = np.ones([train_c,1])
    
#     # create np array of ones for validation 
#     val = np.ones([twenty_percent,1])



result = split_data(parse_stackoverflow_data())

print(result)
# result[0].shape == (52544, 23)
# result[1].shape 
# objects_equal(result[0][:,0].mean(), 0.24120736906211937)

(65679, 23)
None


### Q3 Linear Regression

Now we are going to build a simple scikit-learn-like class for least squares linear regression.  Recall from lecture that the linear regression approach models the data as:
$$ y^{(i)} \approx \theta^T x^{(i)} $$
and the optimal $\theta$ is given by:
$$ \theta^* = (X^T X)^{-1}X^T y $$
using the notation described in the slides and course notes.  Recall, as mentioned in class, that you should use the `np.linalg.solve()` function rather than the `np.linalg.inv()` function to compute this solution.

Implement the class below, plus the `squared_error` function.

In [396]:
def squared_error(y_pred, y):
    """ Utility function to compute squared error
    args:
        y_pred : np.ndarray[num_examples] -- the predictions
        y : np.ndarray[num_examples] -- the ground truth values

    returns:
        float : _average_ squared error between y_pred and y
    """
#     y_pred.astype(int)
#     print(len(y_pred))
    squared_error = 0

    if isinstance(y_pred,float): 
        for i in range(len(y)):   
            squared_error += (y[i] - y_pred) **2

    else:
        for i in range(len(y)):   
            squared_error += (y[i] - y_pred[i]) **2

    avrg_squared_error = squared_error/ len(y)
    
    return avrg_squared_error
    
# @mugrade.local_tests
# @mugrade.submit_tests('28PY3HJpnZGQT8R2VgdC')

class LinearRegression():
    """ Perform linear regression and predict the output on unseen examples. 
    
    attributes: 
        theta (np.ndarray) : vector containing parameters for the features
    """

    def __init__(self, X, y):
        """ Train the linear regression model by computing the estimate of the parameters
        You should store the model parameters in self.theta

        args: 
            X (np.ndarray[num_examples, num_columns]) : matrix of training data
            y (np.ndarray[num_examples]) : vector of output variables

        return: LinearRegression -- returns itself (for convenience)
        """
#         # Calculate the mean of X and y
#         x_mean = np.mean(X)
#         y_mean = np.mean(y)
        
#         # calculate xy cov and x variance 
#         xy_cov = [((X[i] - x_mean) * (y[i] - y_mean)) for i in range(len(y))]
#         xvar = [(X[i]-x_mean)**2 for i in range(len(y))]
        
#         # calculate beta
#         beta = sum(xy_cov)/sum(xvar)
#         alpha = y_mean - (beta * x_mean)
        
        self.theta = np.linalg.solve(X.T @ X, X.T @ y)
    

    def predict(self, X): 
        """ Use the learned model to predict the output of X_p

        args: 
            X : np.ndarray[num_examples, num_columns] -- matrix of features for which we form a prediction

        return: 
            np.ndarray[num_examples], vector of predicted outputs
        """
        ypred = X @ self.theta
    
        return ypred
    
    
    

In [414]:
X_train, y_train, X_val, y_val = split_data(parse_stackoverflow_data())
lin_reg = LinearRegression(X_train, y_train)

# lin_reg.predict(X_val)[:10]
# 
# lin_reg.theta

y_train

array([-1.,  1.,  0., ...,  0., -1., -1.])

## Q4 Evaluation versus baselines

As a final consideration, If you implement this properly, you will see that we get a squared error of around 1.3 on the validation set.  Is this "good"?  This is one of the more subtle points of many data science problems, but we can start to get some sense of this by looking at what the predictions would look like if we _just_ predicted the mean target output on the training set (this is essentially the "simplest" prediction we could make if we didn't look at the features at all).

Implement the following function to evaluate our linear regression.  Think about what this signifies about the quality of the solution.

In [401]:
# @mugrade.local_tests
# @mugrade.submit_tests('28PY3HJpnZGQT8R2VgdC')

def evaluate_linear_regression(X_train, y_train, X_val, y_val):
    """ Evaluate the squared error of linear regression versus the simple mean-prediciton baseline.
    
    Args: X_train, y_train, X_val, y_val -- output of split_data() function
    
    Return: Tuple[validation_mse, baseline_mse]:
        validation_mse: float -- squared error of predictions on validation set, when training on training set
        baseline_mse: float -- squared error of predicting the mean on the training set
    """
    y_val_pred = LinearRegression(X_train, y_train)
    baseline_mse = squared_error(np.average(y_train), y_val)
    validation_mse = squared_error(y_val_pred.predict(X_val), y_val)
    return (validation_mse, baseline_mse)

Submitting tests for function evaluate_linear_regression():
  Test 1 PASSED
